In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

from NormalizingFlows.src.train import train_backward
from NormalizingFlows.src.scores import log_likelihood
from NormalizingFlows.src.utils import update_device, load_best_model, load_checkpoint_model

from NormalizingFlows.src.flows import *
from NormalizingFlows.src.data.density.toydata import ToyDataset

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device_cpu = torch.device("cpu")

In [ ]:
dim_input = 25
dataset = ToyDataset(dim_input=dim_input, samples=1000)

In [ ]:
dim_input = dataset.dim_input
num_trans = 6 #Must be even
perm_types = ['identity', 'alternate','random']
dim_hidden = [100,100,100]
flows, names = [], []
flow_forward=False

In [ ]:
#Linear and mean field

transformations = create_constant_trans(num_trans, dim_input, flow_forward)
mean_field = create_flows_with_identity(dim_input, transformations, flow_forward) 
name = 'Mean field'
flows.append(mean_field), names.append(name)

transformations = create_linear_bias_trans(num_trans, dim_in, forward_flow)
linear = create_flows_with_identity(dim_input, transformations, flow_forward)
name = 'Linear'
flows.append(linear), names.append(name)

In [ ]:
#Affine flows with two block

transformations = create_affine_trans(num_trans, dim_input, flow_forward)
aff_coup_id = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'Affine coupling identity'
flows.append(aff_coup_id), names.append(name)

transformations = create_affine_trans(num_trans, dim_input, flow_forward)
aff_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'Affine coupling alternating'
flows.append(aff_coup_alt), names.append(name)

transformations = create_affine_trans(num_trans, dim_input, flow_forward)
aff_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'Affine coupling random'
flows.append(aff_coup_rand), names.append(name)

In [ ]:
#Affine flows with AR

transformations = create_affine_trans(num_trans, dim_input, flow_forward)
aff_ar_id = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'Affine AR identity'
flows.append(aff_ar_id), names.append(name)

transformations = create_affine_trans(num_trans, dim_input, flow_forward)
aff_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'Affine AR alternate'
flows.append(aff_ar_alt), names.append(name)

transformations = create_affine_trans(num_trans, dim_input, flow_forward)
aff_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'Affine AR random'
flows.append(aff_ar_rand), names.append(name)

In [ ]:
#PiecewiseAffine flows with coupling

transformations = create_piecewise_trans(num_trans, forward_flow)
piec_coup_id = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'PiecewiseAffine coupling identity'
flows.append(piec_coup_id), names.append(name)

transformations = create_piecewise_trans(num_trans, forward_flow)
piec_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'PiecewiseAffine coupling alternate'
flows.append(piec_coup_alt), names.append(name)

transformations = create_piecewise_trans(num_trans, forward_flow)
piec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'PiecewiseAffine coupling random'
flows.append(piec_coup_rand), names.append(name)

In [ ]:
#PiecewiseAffine flows with AR

transformations = create_piecewise_trans(num_trans, forward_flow)
piec_ar_id = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'PiecewiseAffine AR identity'
flows.append(piec_ar_id), names.append(name)

transformations = create_piecewise_trans(num_trans, forward_flow)
piec_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'PiecewiseAffine AR alternate'
flows.append(piec_ar_alt), names.append(name)

transformations = create_piecewise_trans(num_trans, forward_flow)
piec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'PiecewiseAffine AR random'
flows.append(piec_ar_rand), names.append(name)

In [ ]:
#PiecewiseAffineAffine with twoblock

transformations = create_affinepiecewise_trans(num_trans, forward_flow)
piecaf_coup_id = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'PiecewiseAffineAffine coupling identity'
flows.append(piecaf_coup_id), names.append(name)

transformations = create_affinepiecewise_trans(num_trans, forward_flow)
piecaf_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'PiecewiseAffineAffine coupling alternate'
flows.append(piecaf_coup_alt), names.append(name)

transformations = create_affinepiecewise_trans(num_trans, forward_flow)
piecaf_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'PiecewiseAffineAffine coupling random'
flows.append(piecaf_coup_rand), names.append(name)

In [ ]:
#PiecewiseAffineAffine with AR

transformations = create_affinepiecewise_trans(num_trans, forward_flow)
piecaf_ar_id = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'PiecewiseAffineAffine AR identity'
flows.append(piecaf_ar_id), names.append(name)

transformations = create_affinepiecewise_trans(num_trans, forward_flow)
piecaf_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'PiecewiseAffineAffine AR alternate'
flows.append(piecaf_ar_alt), names.append(name)

transformations = create_affinepiecewise_trans(num_trans, forward_flow)
piecaf_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'PiecewiseAffineAffine AR random'
flows.append(piecaf_ar_rand), names.append(name)

In [ ]:
#ContinuousPiecewise with twoblock

transformations = create_continuous_piecewise_trans(num_trans, forward_flow)
conpiec_coup_iden = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'ContinuousPiecewiseAffine coupling identity'
flows.append(conpiec_coup_iden), names.append(name)

transformations = create_continuous_piecewise_trans(num_trans, forward_flow)
conpiec_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'ContinuousPiecewiseAffine coupling alternate'
flows.append(conpiec_coup_alt), names.append(name)

transformations = create_continuous_piecewise_trans(num_trans, forward_flow)
conpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'ContinuousPiecewiseAffine coupling random'
flows.append(conpiec_coup_rand), names.append(name)

In [ ]:
#ContinuousPiecewise with AR

transformations = create_continuous_piecewise_trans(num_trans, forward_flow)
conpiec_ar_iden = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'ContinuousPiecewiseAffine AR identity'
flows.append(conpiec_ar_iden), names.append(name)

transformations = create_continuous_piecewise_trans(num_trans, forward_flow)
conpiec_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'ContinuousPiecewiseAffine AR alternate'
flows.append(conpiec_ar_alt), names.append(name)

transformations = create_continuous_piecewise_trans(num_trans, forward_flow)
conpiec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'ContinuousPiecewiseAffine AR random'
flows.append(conpiec_ar_rand), names.append(name)

In [ ]:
#ContinuousPiecewiseAffine with twoblock

transformations = create_affinecontinuous_trans(num_trans, forward_flow)
affconpiec_coup_iden = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'ContinuousPiecewiseAffineAffine coupling identity'
flows.append(affconpiec_coup_iden), names.append(name)

transformations = create_affinecontinuous_trans(num_trans, forward_flow)
affconpiec_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'ContinuousPiecewiseAffineAffine coupling alternate'
flows.append(affconpiec_coup_alt), names.append(name)

transformations = create_affinecontinuous_trans(num_trans, forward_flow)
affconpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'ContinuousPiecewiseAffineAffine coupling random'
flows.append(affconpiec_coup_rand), names.append(name)

In [ ]:
#ContinuousPiecewiseAffine with AR

transformations = create_affinecontinuous_trans(num_trans, forward_flow)
affconpiec_ar_iden = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
name = 'ContinuousPiecewiseAffineAffine AR identity'
flows.append(affconpiec_ar_iden), names.append(name)

transformations = create_affinecontinuous_trans(num_trans, forward_flow)
affconpiec_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
name = 'ContinuousPiecewiseAffineAffine AR alternate'
flows.append(affconpiec_ar_alt), names.append(name)

transformations = create_affinecontinuous_trans(num_trans, forward_flow)
affconpiec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'ContinuousPiecewiseAffineAffine AR random'
flows.append(affconpiec_ar_rand), names.append(name)

In [ ]:
#Alternating Piecewise Affine with twoblock

transformations = create_alt_piecewise_affine_trans(num_trans, forward_flow)
afpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'Alternating Affin_PiecewiseAffine coupling random'
flows.append(afpiec_coup_rand), names.append(name)

In [ ]:
#Alternating Piecewise Affine with AR

transformations = create_alt_piecewise_affine_trans(num_trans, forward_flow)
afpiec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
name = 'Alternating Affin_PiecewiseAffine AR random'
flows.append(afpiec_ar_rand), names.append(name)

In [ ]:
#Alternating with Linear layer and Affine

transformations = create_alt_linear_affine_trans(num_trans, dim_input, forward_flow)
linaff_coup_iden = create_flows_with_twoblock(dim_input, dim_hidden, num_trans, perm_types[0], flow_forward)
name = 'Alternating Linear_Affine coupling identity'
flows.append(linaff_coup_iden), names.append(name)

transformations = create_alt_linear_affine_trans(num_trans, dim_input, forward_flow)
linaff_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, num_trans, perm_types[2], flow_forward)
name = 'Alternating Linear_Affine coupling random'
flows.append(linaff_coup_rand), names.append(name)

transformations = create_alt_linear_affine_trans(num_trans, dim_input, forward_flow)
linaff_ar_iden = create_flows_with_AR(dim_input, dim_hidden, num_trans, perm_types[0], flow_forward)
name = 'Alternating Linear_Affine AR identity'
flows.append(linaff_ar_iden), names.append(name)

transformations = create_alt_linear_affine_trans(num_trans, dim_input, forward_flow)
linaff_ar_rand = create_flows_with_AR(dim_input, dim_hidden, num_trans, perm_types[2], flow_forward)
name = 'Alternating Linear_Affine AR random'
flows.append(linaff_ar_rand), names.append(name)

In [ ]:
#Various Alternating Linear and Conditioner transformation

transformations = create_alt_linear_piecewise_trans(num_trans, dim_input, forward_flow)
linpiece_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, num_trans, perm_types[2], flow_forward)
name = 'Alternating Linear_PiecewiseAffine coupling random'
flows.append(linpiece_coup_rand), names.append(name)

transformations = create_alt_linear_continuous_trans(num_trans, dim_input, forward_flow)
lincont_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, num_trans, perm_types[2], flow_forward)
name = 'Alternating Linear_ContinuousPiece coupling random'
flows.append(lincont_coup_rand), names.append(name)

transformations = create_alt_linear_affinepiecewise_trans(num_trans, dim_input, forward_flow)
linaffpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, num_trans, perm_types[2], flow_forward)
name = 'Alternating Linear_AffinePiece coupling random'
flows.append(linaffpiec_coup_rand), names.append(name)

transformations = create_alt_linear_affinecontinuous_trans(num_trans, dim_input, forward_flow)
linaffcont_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, num_trans, perm_types[2], flow_forward)
name = 'Alternating Linear_AffineContinuous coupling random'
flows.append(linaffcont_coup_rand), names.append(name)

In [ ]:
for ind, flow in enumerate(flows):
    flow.name = names[ind]

In [ ]:
losses = []
optimizers = []

epochs = 100
batch_size = 32

for i in range(len(flows)):
    flow = flows[i]
    update_device(device, flow, dataset)

    #optimizer = torch.optim.AdamW(flow.parameters(), lr=1e-3, weight_decay=1e-3)
    optimizer = torch.optim.SGD(flow.parameters(), lr=1e-4,  weight_decay=1e-3)
    optimizers.append(optimizer)

    losses.append(train_backward(flow, dataset.get_training_data(), optimizer, epochs, batch_size, print_n=200, save_checkpoint=True, burn_in=-1))

    update_device(device_cpu, flow, dataset)

In [ ]:
best_flows = []
for flow in flows:
    best_flows.append(load_best_model(flow))

In [ ]:
log_scale = False
from_iter = 75

plt.subplot(2,1,1)
for i in range(len(losses)):
    plt.plot(losses[i], label=names[i], alpha=0.8)
plt.legend()

plt.subplot(2,1,2)
for i in range(len(losses)):
    plt.plot((losses[i])[from_iter:], label=names[i], alpha=0.8)
plt.legend()

if log_scale:
    plt.yscale('log')

In [ ]:
print('Results based on training data:' + '\n')

train_data = dataset.get_training_data()
mean_target = torch.mean(dataset.evaluate(train_data)).detach().numpy()
for flow in best_flows:
    log_lik, mean = log_likelihood(train_data, flow)
    print("Mean loglikelihood for {}: {}".format(str(flow), mean))
    print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))
    

print("Mean loglikelihood with actual distribution: {}".format(mean_target))

In [ ]:
print('Results based on test data' + '\n')

test_data = dataset.get_test_data()
mean_target = torch.mean(dataset.evaluate(test_data)).detach().numpy()
for flow in best_flows:
    log_lik, mean = log_likelihood(test_data, flow)
    print("Mean loglikelihood for {}: {}".format(str(flow), mean))
    print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))
    

print("Mean loglikelihood with actual distribution: {}".format(mean_target))

In [ ]:
print('Results based on new sample from each flow:' + '\n')

for flow in best_flows[:2]:
    with torch.no_grad():
        sample, log_prob = flow.sample(800)
        sample_last = sample[-1]

    log_lik, mean = log_likelihood(sample_last, flow)
    print("Mean loglikelihood for {}: {}".format(str(flow), mean))
    print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))
    
    mean_target = torch.mean(dataset.evaluate(sample_last)).detach().numpy()
    print("Mean loglikelihood with actual distribution: {}".format(mean_target))   